In [1]:
import json
import numpy as np
import pdb
import itertools
import ast

In [3]:
num_gpu = 8
config = {0: [7],
          1: [4,2,1],
          2: [4,1,1,1],
          3: [3,3],
          4: [3,2,1],
          5: [3,1,1,1],
          6: [2,2,3],
          7: [2,1,1,3],
          8: [1,1,1,1,3],
          9:[2,2,2,1],
          10:[2,1,1,2,1],
          11:[2,1,1,1,1,1],
          12:[1,1,1,1,1,1,1]
        }

# with open('partition_code.json', 'w') as f:
#     json.dump(config, f, indent=4)
    
def mapping(seq):
    map_dict = {7: 0, 4: 0, 3: 0, 2: 0, 1: 0}
    for s in seq:
        c = config[s]
        for item in c:
            map_dict[item] += 1
    return str(map_dict)    

### 1 GPU base case

In [17]:
def map_nonrecurse(num_gpu):
    mapped_dict = {}
    gen = itertools.product(range(len(config)),repeat=num_gpu)

    for seq in gen:
        mapping_str = mapping(seq)
        if mapping_str not in mapped_dict:
            mapped_dict[mapping_str] = str(list(seq))
    return mapped_dict

In [18]:
mapped_1gpu = map_nonrecurse(1)
with open('mapped_1gpu.json', 'w') as f:
    json.dump(mapped_1gpu, f, indent=4)

In [19]:
def map_recurse(num_gpu):
    if num_gpu == 1:
        with open('mapped_1gpu.json') as f:
            map_space = json.load(f)
        return map_space
    else:
        prev_space = map_recurse(num_gpu-1)
        map_space = {}
        for key, space_actual in prev_space.items():       
            for config_key, partition in config.items(): # [4,2,1]
                key_dict = ast.literal_eval(key) # {7: 1, 4: 0, 3: 0, 2: 0, 1: 0}
                space_actual_list = ast.literal_eval(space_actual)
                for mig_slice in partition: # 4
                    key_dict[mig_slice] += 1
                # key_dict = {7: 1, 4: 1, 3: 0, 2: 1, 1: 1}
                if str(key_dict) not in map_space:
                    space_actual_list.append(config_key)
                    map_space[str(key_dict)] = str(space_actual_list)
                
        return map_space

In [20]:
map_recurse(2)

{'{7: 2, 4: 0, 3: 0, 2: 0, 1: 0}': '[0, 0]',
 '{7: 1, 4: 1, 3: 0, 2: 1, 1: 1}': '[0, 1]',
 '{7: 1, 4: 1, 3: 0, 2: 0, 1: 3}': '[0, 2]',
 '{7: 1, 4: 0, 3: 2, 2: 0, 1: 0}': '[0, 3]',
 '{7: 1, 4: 0, 3: 1, 2: 1, 1: 1}': '[0, 4]',
 '{7: 1, 4: 0, 3: 1, 2: 0, 1: 3}': '[0, 5]',
 '{7: 1, 4: 0, 3: 1, 2: 2, 1: 0}': '[0, 6]',
 '{7: 1, 4: 0, 3: 1, 2: 1, 1: 2}': '[0, 7]',
 '{7: 1, 4: 0, 3: 1, 2: 0, 1: 4}': '[0, 8]',
 '{7: 1, 4: 0, 3: 0, 2: 3, 1: 1}': '[0, 9]',
 '{7: 1, 4: 0, 3: 0, 2: 2, 1: 3}': '[0, 10]',
 '{7: 1, 4: 0, 3: 0, 2: 1, 1: 5}': '[0, 11]',
 '{7: 1, 4: 0, 3: 0, 2: 0, 1: 7}': '[0, 12]',
 '{7: 0, 4: 2, 3: 0, 2: 2, 1: 2}': '[1, 1]',
 '{7: 0, 4: 2, 3: 0, 2: 1, 1: 4}': '[1, 2]',
 '{7: 0, 4: 1, 3: 2, 2: 1, 1: 1}': '[1, 3]',
 '{7: 0, 4: 1, 3: 1, 2: 2, 1: 2}': '[1, 4]',
 '{7: 0, 4: 1, 3: 1, 2: 1, 1: 4}': '[1, 5]',
 '{7: 0, 4: 1, 3: 1, 2: 3, 1: 1}': '[1, 6]',
 '{7: 0, 4: 1, 3: 1, 2: 2, 1: 3}': '[1, 7]',
 '{7: 0, 4: 1, 3: 1, 2: 1, 1: 5}': '[1, 8]',
 '{7: 0, 4: 1, 3: 0, 2: 4, 1: 2}': '[1, 9]',
 '{7: 0

In [21]:
map_8 = map_recurse(8)
with open('mapped_8gpu.json', 'w') as f:
    json.dump(map_8, f, indent=4)

In [22]:
map_15 = map_recurse(15)
with open('mapped_15gpu.json', 'w') as f:
    json.dump(map_15, f, indent=4)

In [25]:
len(map_8)

8217

In [26]:
len(json_8gpu)

8217